In [ ]:
import os
import lib
import numpy as np
import pandas as pd

DATA_DIR = os.path.join(lib.basic.DATA_DIR(), 'myelin_grad')

In [ ]:
regions = ['cb', 'cc', 'sc']
map_funcs = [lib.myelin.get_cb_map, lib.myelin.get_cc_map, lib.myelin.get_sc_map]

In [ ]:
# Generating myelination maps
maps = {region: lib.myelin.get_maps_of_all_subs(map_func) for region, map_func in zip(regions, map_funcs)}

# To pickle file
for region in maps:
    maps[region].to_pickle(os.path.join(DATA_DIR, f'{region}_maps.pickle'))

In [ ]:
# Loading myelination maps
maps = {region: pd.load_pickle(os.path.join(DATA_DIR, f'{region}_maps.pickle')) for region in regions}

# Gradient analysis
basic_grad_results, time_profiles = {}, {}
for region in regions:
    _, grad_pca = lib.grad.basic_grad_pca(maps[region])
    basic_grad_results['region'] = grad_pca
    time_profiles['region'] = lib.grad.get_time_profile(basic_grad_results[region][1])

In [ ]:
# Plots
for region in regions:
    lib.grad.plot_time_profile(time_profiles[region], n_comp=3, roi_name=region.upper())

In [1]:
# Quantative comparison on PC1
import scipy.stats as stats

def corr_func(arr1, arr2, alpha=0.05):
    _arr1, _arr2 = arr1.flatten(), arr2.flatten()
    assert len(_arr1) == len(_arr2)
    length = len(_arr1)
    
    deltas = np.arange(4 - length, length - 4 + 1)
    n_idx = len(deltas)
    corrs, ci_lbs, ci_ubs = np.ones([n_idx]), np.ones([n_idx]), np.ones([n_idx])
    for i, delta in enumerate(deltas):
        _delta = np.abs(delta)
        n = length - _delta
        if delta <= 0:
            r, sig = stats.pearsonr(_arr1[:n], _arr2[_delta:])
        else:
            r, sig = stats.pearsonr(_arr2[:n], _arr1[_delta:])
        
        zr = np.log((1 + r) / (1 - r)) / 2
        zlb = zr - stats.norm.isf(alpha / 2) / (n - 3) ** 0.5
        zub = zr + stats.norm.isf(alpha / 2) / (n - 3) ** 0.5
        corrs[i] = r
        ci_lbs[i] = (np.exp(2 * zlb) - 1) / (np.exp(2 * zlb) + 1)
        ci_ubs[i] = (np.exp(2 * zub) - 1) / (np.exp(2 * zub) + 1)
        print(i, r, zr, delta)

    return corrs, ci_lbs, ci_ubs

def plot_corr_func(arr):
    pass

In [ ]:
corr_func(np.array([1, 3, 5, 5, 6, 5]), np.array([3, 5, 6, 7, 3, 8]))